<a href="https://colab.research.google.com/github/elarix0404-svs/anti-sleep/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
int sensorPin = 2;
int buzzerPin = 8;
int ledPin = 9;

void setup() {
  pinMode(sensorPin, INPUT);
  pinMode(buzzerPin, OUTPUT);
  pinMode(ledPin, OUTPUT);
}

void loop() {
  int eyeStatus = digitalRead(sensorPin);

  if (eyeStatus == LOW) { // Eye closed
    delay(2000); // Wait 2 sec to confirm sleepiness
    if (digitalRead(sensorPin) == LOW) {
      digitalWrite(buzzerPin, HIGH);
      digitalWrite(ledPin, HIGH);
    }
  } else {
    digitalWrite(buzzerPin, LOW);
    digitalWrite(ledPin, LOW);
  }
}


In [ ]:
import cv2
import numpy as np
from keras.models import load_model
import winsound  # For buzzer sound (Windows)

# Load pre-trained eye state model
model = load_model("drowsiness_model.h5")

# Start camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (64,64)) / 255.0
        roi = roi.reshape(1,64,64,1)

        prediction = model.predict(roi)
        if prediction[0][0] > 0.5:   # Suppose [0]=Drowsy, [1]=Awake
            cv2.putText(frame, "DROWSY", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            winsound.Beep(1000, 500)  # Alarm sound
        else:
            cv2.putText(frame, "AWAKE", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
